# 화면에서 움직이는 개체 감지

이 문서는 openCV를 기반으로 프레임에 이동 또는 변경하는 개체가 있는지 여부를 분석하는 데 사용됩니다.


# 카메라 기능 라이브러리 가져오기

다음 코드 블록을 실행한 후 잠시 기다렸다가 카메라가 초기화될 때까지 기다립니다. 초기화가 성공하면 코드 블록 아래에 300x300 크기의 실시간 비디오 화면이 나타납니다.

이 화면을 마우스 오른쪽 단추로 클릭한 후 출력용 새 보기 만들기를 클릭하면 다시 창에 카메라 화면을 넣을 수 있습니다. 문서의 다른 부분을 탐색하더라도 언제든지 카메라 화면을 볼 수 있습니다. 이 방법은 다른 위젯에도 적용됩니다.

이 코드 블록을 여러 번 실행하면 초기화에 실패할 수 있습니다. 이 솔루션은 이미 jetbot.Camera에 포함되어 있습니다. 커널을 다시 시작하기만 하면 되지만 탭 위의 원 화살표를 사용하지 않도록 주의하십시오. 그래도 초기화에 실패할 수 있습니다.

커널을 재시작하는 것은 권장되는 방법입니다:
왼쪽에 있는 파일 브라우저에서 녹색 점이 앞에 있는*.ipynb 파일을 마우스 오른쪽 버튼으로 클릭하고(녹색 점은 커널이 실행 중임을 나타냅니다), Shut Down Kernel을 선택하면 녹색 점이 사라진 다음 이 탭을 닫고 방금 닫았던*.ipynb 파일을 더블 클릭하여 커널을 다시 시작합니다.

다음 코드를 다시 실행하면 카메라가 정상적으로 초기화됩니다.

In [1]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

# 동작감지기능

움직임 감지 기능은 openCV를 기반으로 합니다. OpenCV는 Jetpack에 미리 설치되어 있으므로 다음 코드 블록을 실행하여 필요한 기능 라이브러리를 직접 가져올 수 있습니다. Jetpack을 사용하지 않는 경우 openCV 또는 imutils를 단말기에 수동으로 설치하고 sudo pip3 install openCV-python'과 sudo pip3 install imutils를 사용하여 각각 라이브러리를 설치해야 할 수 있습니다. 두 라이브러리가 누락되었음을 알리는 오류가 없으면 이를 무시하고 바로 진행하면 됩니다. 다음 코드 블록이 실행됩니다.

In [2]:
import cv2
import imutils
import datetime

# avg is used to save a frame of reference picture (background)
# the new picture is compared with it to determine where in the picture has changed.
avg = None


lastMovtionCaptured = datetime.datetime.now()

# Motion detection function
def motionDetect(imgInput):
    global avg, lastMovtionCaptured
    
    # Get the current timestamp.
    timestamp = datetime.datetime.now()
    
    # Convert the frame to black and white, which can increase the efficiency of analysis.
    gray = cv2.cvtColor(imgInput, cv2.COLOR_BGR2GRAY)
    
    # Gaussian blur the frame to avoid misjudgment caused by noise.
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # If the reference frame (background) has not been obtained, create a new one.
    if avg is None:
        avg = gray.copy().astype("float")
        return imgInput

    # background update.
    cv2.accumulateWeighted(gray, avg, 0.5)
    
    # Compare the difference between the new frame and the background.
    frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))

    # Get the outline of the changed area in the frame.
    thresh = cv2.threshold(frameDelta, 5, 255,
        cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # There may be more than one area changes in the frame, so you need to use a for loop to get all the contours.
    for c in cnts:
        # The default here is 30, which is the threshold of the change area. We only analyze the area greater than 800.
        # The smaller the value, the more sensitive the motion detection, but it may also detect meaningless noise.
        if cv2.contourArea(c) < 30:
            continue

        # Draw elements, including rectangle and text.
        (mov_x, mov_y, mov_w, mov_h) = cv2.boundingRect(c)
        cv2.rectangle(imgInput, (mov_x, mov_y), (mov_x+mov_w, mov_y+mov_h), (128, 255, 0), 1)

        # Save the current timestamp to mark the time when the change is detected.
        lastMovtionCaptured = timestamp

    # In order to avoid the high flickering frequency of drawing elements
    # within 0.5 seconds after the motion ends, elements stay.
    if (timestamp - lastMovtionCaptured).seconds >= 0.5:
        cv2.putText(imgInput,"Motion Detecting",(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(128,255,0),1,cv2.LINE_AA)
    else:
        cv2.putText(imgInput,"Motion Detected",(10,80), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,128,255),1,cv2.LINE_AA)
    
    # Return to the processed frame.
    return imgInput

# 비디오 프레임 및 디스플레이 처리

다음 코드를 실행하면 프레임의 색상이 변경되어 'motionDetect()' 기능에 의해 비디오 화면이 성공적으로 처리되었음을 알 수 있습니다.

In [3]:
def execute(change):
    global image_widget
    image = change['new']
    image_widget.value = bgr8_to_jpeg(motionDetect(image))
    
execute({'new': camera.value})
camera.unobserve_all()
camera.observe(execute, names='value')

이 시점에서 당신은 모든 코드를 실행한 것입니다. 객체가 프레임에서 움직이거나 변경되면 텍스트 내용이 변경되고 녹색 직사각형이 변경된 영역을 표시합니다.

# 이 처리를 끄고 카메라를 중지합니다
다음 코드를 실행하여 이미지 처리 기능을 끕니다.

In [4]:
camera.unobserve(execute, names='value')

다시 말씀드리지만, 나중에 노트북에 있는 카메라를 사용할 수 있도록 카메라 커넥터를 제대로 닫도록 하겠습니다.

In [4]:
camera.stop()